In [217]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.stem import SnowballStemmer,PorterStemmer
from stemming import lovins,porter2,paicehusk
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import stop_words
import pprint
import re
import time
import spacy
import datetime
import requests
import BeautifulSoup
import json
%matplotlib inline

In [2]:
url='https://en.wikipedia.org/wiki/Wikipedia:List_of_controversial_issues'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
soup=soup.findAll('div',{'class':'div-col columns column-count column-count-2'})
controversial_terms=[]
for i in range(len(soup)):
    controversial_terms.extend([x.text for x in soup[i].findAll('li')])

In [3]:
url='http://www.conservapedia.com/Essay:Best_New_Conservative_Words'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
conservative_terms=[]
for i in range(1,len(soup.findAll('table')[1].findAll('tr'))):
    conservative_terms.append(soup.findAll('table')[1].findAll('tr')[i].td.text)

In [4]:
technical_terms=[]

In [5]:
url='http://www.math.utah.edu/~wisnia/glossary.html'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('h3'):
    technical_terms.append(x)

In [6]:
url='https://en.wikipedia.org/wiki/Glossary_of_physics'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
soup=soup.findAll('ul')
for x in soup[1:-37]:
    for y in x.findAll('a'):
        technical_terms.append(y.text)

In [7]:
url='https://en.wikipedia.org/wiki/Glossary_of_chemistry_terms'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
soup=soup.findAll('ul')
for x in soup[3:-17]:
    for y in x.findAll('a'):
        technical_terms.append(y.text)

In [8]:
url='https://en.wikipedia.org/wiki/Glossary_of_biology'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
soup=soup.findAll('ul')
for x in soup[2:-15]:
    for y in x.findAll('a'):
        technical_terms.append(y.text)

In [9]:
url='https://en.wikipedia.org/wiki/Glossary_of_philosophy'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
soup=soup.findAll('a')
for x in soup[70:-87]:
    technical_terms.append(x.text)

In [10]:
url='http://members.cruzio.com/~zdino/psychology/political.science.glossary.htm'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
soup=soup.findAll('dl')
for x in soup:
    for y in x.findAll('a'):
        if len(y.text)>1:
            technical_terms.append(y.text)

In [20]:
url='http://www.hermes-press.com/psgloss1.htm'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
soup=soup.findAll('p')
for x in soup:
    for y in x.text.split('.'):
        for z in y.split('-'):
            if len(z)<15 and len(z)>5:
                if 'U' not in z:
                    z=z.strip(')').strip(', ').strip('(')
                    technical_terms.append(z)

In [21]:
theological=[]
url='http://www.rebecca-writes.com/theological-terms-in-ao/'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('a')[22:-2]:
    theological.append(x.text)

In [74]:
sports=[]

In [75]:
url='https://en.wikipedia.org/wiki/Glossary_of_American_football'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('dt')[:-1]:
    sports.append(x.text)

In [76]:
url='http://www.socceramerica.com/glossary/'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('strong')[19:]:
    sports.append(x.text)

In [77]:
url='https://en.wikipedia.org/wiki/Glossary_of_professional_wrestling_terms'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('dl')[::2]:
    sports.append(x.text)

In [78]:
url='http://wrestling.isport.com/wrestling-guides/wrestling-glossary/?key=1831'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('strong'):
    sports.append(x.text)

In [79]:
url='https://en.wikipedia.org/wiki/Glossary_of_motorsport_terms'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('dt'):
    sports.append(x.text)

In [80]:
url='https://en.wikipedia.org/wiki/Glossary_of_basketball_terms'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('dt'):
    if '&#160;' in x.text:
        sports.append(x.text[:-6])
    else:
        sports.append(x.text)

In [87]:
fitness=[]

In [88]:
url='http://www.bodybuilding.com/fun/glossam.htm'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('b'):
    fitness.append(x.text)

In [89]:
url='http://www.bodybuilding.com/fun/glossnz.htm'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('strong'):
    fitness.append(x.text)

In [103]:
anarchy=[]
url='https://en.wikipedia.org/wiki/Glossary_of_anarchism'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('dfn'):
    anarchy.append(x.text)

In [110]:
marxism=[]
for letter in 'abcdefghijklmnopqrstuvwxyz':
    baseurl='https://www.marxists.org/glossary/'+letter+'.htm'
    r=requests.get(baseurl)
    soup=BeautifulSoup.BeautifulSoup(r.text)
    for x in soup.findAll('a'):
        if '#8212;' in x.text:
            x.text.replace('#8212;',"-")
        if '&#8217;' in x.text:
            x.text.replace('&#8217;',"'")
        if '&#8211;' in x.text:
            x.text.replace('&#8211;',"'")
        marxism.append(x.text)

In [118]:
drugs=[]
for letter in 'abcdefghijklmnopqrstuvwxyz':
    url='http://www.noslang.com/drugs/dictionary/'+letter+'/'
    r=requests.get(url)
    soup=BeautifulSoup.BeautifulSoup(r.text)
    for x in soup.findAll('b')[5:]:
        drugs.append(x.text)

In [125]:
gaming=[]

In [126]:
url='http://www.theaveragegamer.com/terminology/'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('strong'):
    gaming.append(x.text[:-1])

In [127]:
url='https://en.wikipedia.org/wiki/Category:Video_game_terminology'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('a')[4:]:
    x=x.text
    gaming.append(x)

In [152]:
url='http://videogamecritic.com/extras/glossary.htm'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('b')[:-1]:
    gaming.append(x.text)

In [197]:
environmental=[]

In [198]:
url='http://www.theconscientioushome.net/glossary.php'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('a')[4:-26]:
    environmental.append(x.text)

In [199]:
url='https://en.wikipedia.org/wiki/Glossary_of_environmental_science'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('a')[36:-134]:
    if '[' not in x.text:
        if 'edit' not in x.text:
            environmental.append(x.text)

In [260]:
weeaboo=[]
url='http://imgur.com/gallery/c4oKNNO'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('div',{'class':'post-image-description'})[0]:
    x=x.split('\n')
    for y in x[1:-5]:
        weeaboo.append(y.split(' =')[0])

In [34]:
liberal_string="""Ally: Someone who advocates for and supports members of a community other than their own.
Reaching across differences to achieve mutual goals.
Androgynous: Someone who reflects an appearance that is both masculine and feminine, or who
appears to be neither or both a boy and a girl.
Asexual: Having no evident sex or sex organs. In usage, may refer to a person who is not sexually
active, or not sexually attracted to other people.
Bias: Prejudice; an inclination or preference, especially one that interferes with impartial judgment.
Bigendered: Refers to those who feel they have both a male and a female side to their
personalities. Some “bigendered” people crossdress, others may eventually have a sex-change
operation, others may do neither.
Biphobia: The irrational fear and intolerance of people who are bisexual.
Birth Sex/Sex: The sex one is assigned at birth due to the presence of whatever external sex
organs. Once this determination is made, it becomes a label used for raising the child in either one
gender image or other (either as male or female).
Bisexual: Also bi. A person who is attracted to two sexes or two genders, but not necessarily
simultaneously or equally. [This used to be defined as a person who is attracted to both genders or
both sexes, but since there are not only two sexes (see intersex and transsexual) and there are not
only two genders (see transgender), this definition is inaccurate.]
Coming out: To recognize one’s sexual orientation, gender identity, or sex identity, and to be
open about it with oneself and with others.
Cross Living: Living full-time in the preferred gender image, opposite to one’s assigned sex at
birth, generally in preparation for a sex change operation.
Direction: Refers to the way in which one is crossing the gender line. Masculine/Male to
Feminine/Female (MTF) is one way; Feminine/ Female to Masculine/Male (FTM) is another. 
 2
Discrimination: The act of showing partiality or prejudice; a prejudicial act.
Domestic Partner: One who lives with their beloved and/or is at least emotionally and financially
connected in a supportive manner with another. Another word for spouse, lover, significant other,
etc.
Dominant culture: The cultural values, beliefs, and practices that are assumed to be the most
common and influential within a given society.
Drab: Means dressing as a boy, referring to men’s clothes or wearing men’s clothes; is used
mainly by gender benders and cross-dressers of both directions. Drag: The act of dressing in
gendered clothing as part of a performance. Drag Queens perform in highly feminine attire. Drag
Kings perform in highly masculine attire. Drag may be performed as a political comment on
gender, as parody, or simply as entertainment. Drag performance does not indicate sexuality,
gender identity, or sex identity.
Family: Colloquial term used to identify other LGBTIQ community members. For example, an
LGBTIQ person saying, “that person is family” often means that the person they are referring to is
LGBTIQ as well.
Family of choice (chosen family): Persons or group of people an individual sees as significant in
his or her life. It may include none, all, or some members of his or her family of origin. In addition,
it may include individuals such as significant others, domestic partners, friends, and coworkers.
FTM: female-to-male (transvestite or transsexual).
Gay: Men attracted to men. Colloquially used as an umbrella term to include all LGBTIQ people.
Gender: 1) A socially constructed system of classification that ascribes qualities of masculinity
and femininity to people. Gender characteristics can change over time and are different between
cultures. Words that refer to gender include: man, woman, transgender, masculine, feminine, and
gender queer. 2) One’s sense of self as masculine or feminine regardless of external genitalia. 3)
Refers to the way we perceive certain things to be masculine or feminine. These things need not be
human; for example, in the language of many cultures, cups are feminine, and pencils masculine.
[Traditionally, we tend to associate gender – that is, masculine and feminine meanings – with
features that include: · physical sex or genitals; · other physical features (for example height,
weight, and body hair); · sexual orientation (gay men are often considered more feminine than their
heterosexual counterparts, and lesbian women more masculine); and · behavior or dress (a man
who cries may be considered unmanly, a woman who is aggressive or wears a suit-coat and slacks
may be considered unfeminine).]
Gender Binary System: A system of oppression that requires everyone to be raised either male or
female, and masculine or feminine. Eliminates 5 the possibility for other gender expressions, and
gives power to people whose genders do not break gender norms at the expense of transgender and
intersex people. Manifests itself as transphobia. 
 3
Gender Characteristics: Refers to the primary and secondary sexual physical characteristics like
height, weight, and body hair, over which the individual has no control and which do not constitute
part of their expression or identification. Examples might include a man with a high voice, a
woman with prominent facial hair, or a person with anomalous genitalia (more correctly referred to
as “intersex”).
Gender Conformity: When your gender identity and sex “match” (i.e. fit social norms). For
example, a male who is masculine and identifies as a man.
Gender Expression/Gender Image: The way one presents oneself to the world, as either
masculine or feminine, or both or neither. This can include dress, posture, vocal inflection, and
other behavior.
Gender Identity: 1) “Gender identity” refers to an individual’s self-awareness or fundamental
sense of themselves as being masculine or feminine, and male or female. The phrase “gender
identity” originated as a psychiatric term, and is commonly used to protect transsexual or
transgender employees, particularly those who transition from one sex to another on the job. 2)
The gender that a person sees oneself as. This can include refusing to label oneself with a gender.
Gender identity is also often conflated with sexual orientation, but this is inaccurate. Gender
identity does not cause sexual orientation. For example, a masculine woman is not necessarily a
lesbian.
Genderism: Holding people to traditional expectations based on gender, or punishing or excluding
those who don’t conform to traditional gender expectations.
Gender-neutral: Nondiscriminatory language to describe relationships—e.g. “spouse” and
“partner” are gender-neutral alternatives to the gender-specific words “husband,” “wife,”
“boyfriend” and “girlfriend.”
Gender Queer (or Genderqueer): A person who redefines or plays with gender, or who refuses
gender altogether. A label for people who bend/break the rules of gender and blur the boundaries.
Gender Role: How “masculine” or “feminine” an individual acts. Societies commonly have norms
regarding how males and females should behave, expecting people to have personality
characteristics and/or act a certain way based on their biological sex.
Gender Stereotypes: “Gender stereotypes” are the patterns or mental templates for what we
expect members of each sex to be. For instance, the stereotype for males frequently includes being
tall, muscular, hirsute, solitary, and impassive. For females it might include being small, weak,
social, sensitive, and emotional.
Gender-variant / Gender non-conforming: Displaying gender traits that are not normatively
associated with their biological sex. “Feminine” behavior or appearance in a male is gender-variant
as is “masculine” behavior or appearance a female. Gender-variant behavior is culturally specific.
 4
Genetic: refers to the chromosomal endowment of the individual, with emphasis on the sex
chromosomes (XX in women and XY in men).
Hate crime: Hate crime legislation often defines a hate crime as a crime motivated by the actual or
perceived race, color, religion, national origin, ethnicity, gender, disability, or sexual orientation of
any person.
Heterosexism: Assuming every person to be heterosexual therefore marginalizing persons who do
not identify as heterosexual. It is also believing heterosexuality to be superior to homosexuality
and all other sexual orientations.
Heterosexuality: Sexual, emotional, and/or romantic attraction to a sex other than your own.
Commonly thought of as “attraction to the opposite sex” but since there are not only two sexes (see
intersex and transsexual), this definition is inaccurate.
Heterosexual Privilege: Benefits derived automatically by being (or being perceived as)
heterosexual that are denied to homosexuals, bisexuals, and queers.
Homophobia: The irrational fear and intolerance of people who are homosexual or of homosexual
feelings within one’s self. This assumes that heterosexuality is superior.
Homosexuality: Sexual, emotional, and/or romantic attraction to the same sex.
Institutional Oppression: Arrangement of a society used to benefit one group at the expense of
another through the use of language, media education, religion, economics, etc.
Internalized Oppression: The process by which an oppressed person comes to believe, accept, or
live out the inaccurate stereotypes and misinformation about their group.
Intersex: Intersexuality is a set of medical conditions that feature congenital anomaly of the
reproductive and sexual system. That is, intersex people are born with “sex chromosomes,”
external genitalia, or internal reproductive systems that are not considered “standard” for either
male or female. The existence of intersexuals shows that there are not just two 10 sexes and that
our ways of thinking about sex (trying to force everyone to fit into either the male box or the
female box) is socially constructed. About 1 in 2000 infants born are at risk for intersex genital
mutilation. An even higher proportion of the population is intersex in some way. This word
replaces “hermaphrodite”, which is generally considered impolite and/or derogatory.
In the closet: Keeping one’s sexual orientation and/or gender or sex identity a secret.
Invisible minority: A group whose minority status is not always immediately visible, such as
some disabled people and LGBTIQ people. This lack of visibility may make organizing for rights
difficult.
Lambda: The Gay Activist Alliance originally chose the lambda, the Greek letter “L”, as a symbol
in 1970. Organizers chose the letter “L” to signify liberation. The word has become a way of 
 5
expressing the concept “lesbian and gay male” in a minimum of syllables and has been adopted by
such organizations as Lambda Legal Defense and Education Fund.
Lesbian: A woman attracted to women.
LGBTIQ: Lesbian, Gay, Bisexual, Transgender, Intersex, Queer.
Male supremacy: A system of oppression that gives power to men and values masculinity, at the
expense of women and femininity.
Marginalized: Excluded, ignored, or relegated to the outer edge of a group/society/community.
Men who have sex with men (MSM): Men who engage in same-sex behavior, but who may not
necessarily self-identify as gay.
MTF: Male to Female (transvestite or transsexual).
Non-Op: Refers to transsexuals who seek sex reassignment through hormones and who crosslive,
but stop just short of surgery. Some have concerns about major surgery, which is not always
successful, others are unable to pay for the expensive procedures surgery would entail, and still
others feel that they are complete without the surgery.
On T: When a FTM takes the hormone testosterone.
Oppression: Results from the use of institutional power and privilege where one person or group
benefits at the expense of another. Oppression is the use of power and the effects of domination.
Out or Out of the closet: Refers to varying degrees of being open about one’s sexual orientation
and/or sex identity or gender identity.
Pansexual: A person who is fluid in sexual orientation and/or gender or sex identity.
Pass: Means to be in your preferred gender image and to be able to do so convincingly in the eyes
of those around you, for example an FTM or cross dresser or drag king who looks like a man and
not like a woman.
Polyamory: The practice of having multiple open, honest love relationships.
Post-Op: A transsexual who has had their sex change operation and now has the physical anatomy
which mimics that of the sex they have transitioned to.
(Institutional) Power: Means control, access and influence. In U.S. society, power means having
control of and access to ruling institutions; freedom from the threat of being questioned or
reprimanded for wrong-doing; and the ability to define standards and norms.
Present: Refers to gender expression and the process of reflecting ones gender to others; someone
who is bigendered may present as female one day and male the next. 
 6
Pre-Op: A transsexual who has not yet had their sex change operation(s) but who ‘plans on having
it/them.
Privilege: Means a right, a favor, an immunity or an advantage specially granted to one individual
or group and withheld from another.
Queer: 1) An umbrella term used to refer to all LGBTIQ people. 2) A political statement, as well
as a sexual orientation, which advocates breaking binary thinking and seeing both sexual 14
orientation and gender identity as potentially fluid. 3) A simple label to explain a complex set of
sexual behaviors and desires. For example, a person who is attracted to multiple genders may
identify as queer. Many older LGBT people feel the word has been hatefully used against them for
too long and are reluctant to embrace it. 4) Originally a synonym for “odd”, this word became a
derogatory expression for gays in the 20th Century. Even though many people still use “queer” as
an anti-gay epithet, a movement emerged in the 1980s that calls itself queer. Used in this way,
queer means sexually dissident, but not necessarily gay. Many gays, transsexuals, bisexuals and
even heterosexuals whose sexuality doesn’t fit into the cultural standard of monogamous
heterosexual marriage have adopted the “queer” label. In academic circles, the term “queer” often
refers to the approaches and sensibilities of queer theory.
Racism: Discrimination against people of color that results from the white supremacy system of
domination. Racism is prejudice plus institutional power.
Rainbow Flag: The Rainbow Freedom Flag was designed in 1978 by Gilbert Baker to designate
the great diversity of the LGBTIQ community. It has been recognized by the International Flag
Makers Association as the official flag of the LGBTIQ civil rights movement.
Self-Identify: Refers to the process of people choosing with which identifying terms/groups they
identify. (E.g. Someone could self-identify as male, female or bigendered, multi-racial, etc.)
Sex: Refers to a person based on their anatomy (external genitalia, chromosomes, and internal
reproductive system). Sex terms are male, female, transsexual, and intersex. Sex is biological,
although social views and experiences of sex are cultural.
Sex Identity: The sex that a person sees themselves as. This can include refusing to label oneself
with a sex.
Sexism: Discrimination against women that results from the male supremacy system of oppression.
Sex-Reassignment Surgery (SRS): Sex change operation.
Sexual minority: 1) Refers to members of sexual orientations or who engage in sexual activities
that are not part of the mainstream. 2) Refers to members of sex groups that do not fall into the
majority categories of male or female, such as intersexuals and transsexuals. 
 7
Sexual Orientation: The deep-seated direction of one’s sexual (erotic) attraction. It is on a
continuum and not a set of absolute categories, sometimes referred to as affection orientation or
sexuality.
Socialized: Refers to the “training” process that takes place once birth sex is determined/decided
upon. People whose birth sex is female are “socialized” as women although they may or may not
self-identify as women.
SRS: Acronym for Sexual Reassignment Surgery, the surgery done by transsexuals to make their
bodies and their sex identity match.
Stereotype: An exaggerated oversimplified belief about an entire group of people without regard
for individual differences.
Straight: Person who is attracted to a gender other than their own. [Commonly thought of as
“attraction to the opposite gender,” but since there are not only two genders (see transgender),
this definition is inaccurate.]
Systems of oppression: Interlocking societal, economic, moral, and religious values that keep
many groups of people down to ensure the power and advantage of a few groups or one group of
people. Some systems of oppression in the US and Europe include: heterosexism, male supremacy,
white supremacy and the gender binary system.
Trans Female/Woman: A male-to-female transition (MTF). [The medical literature tends to use
the extremely demeaning term “male transsexual” to mean the same thing. Note that you can tell
the preferred form is in use when the gender word comes after the “T” word.]
Transgender: 1) Transgender (sometimes shortened to trans or TG) people are those whose
psychological self (“gender identity”) differs from the social expectations for the physical sex they
were born with. To understand this, one must understand the difference between biological sex,
which is one’s body (genitals, chromosomes, ect.), and social gender, which refers to levels of
masculinity and femininity. Often, society conflates sex and gender, viewing them as the same
thing. But, gender and sex are not the same thing. Transgender people are those whose
psychological self (“gender identity”) differs from the social expectations for the physical sex they
were born with. For example, a female with a masculine gender identity or who identifies as a man.
2) An umbrella term for transsexuals, cross-dressers (transvestites), transgenderists, gender queers,
and people who identify as neither female nor male and/or as neither a man or as a woman.
Transgender is not a sexual orientation; transgender people may have any sexual orientation. It is
important to acknowledge that while some people may fit under this definition of transgender, they
may not identify as such.
Transgenderist: A person who lives either full time, or most of the time, in a gender role different
than the role associated with their biological or chromosomal sex (a gender nonconformist).
Transition: A complicated, multi-step process that can take years as transsexuals align their
anatomy with their sex identity; this process may ultimately include sex reassignment surgery
(SRS). 
 8
Trans Man/Male: A female-to-male transition (FTM). [The medical literature tends to use the
extremely demeaning term “female transsexual” to mean the same thing. Note that you can tell the
preferred form is in use when the gender word comes after the “T” word.]
Transphobia: 1) Discrimination, fear or hatred of people who blur traditional gender lines that
results from the gender binary system. Often comes from one’s own insecurity about being a “real
man,” or a “real woman.” 2) Fear or hatred of transgender people; transphobia is manifested in a
number of ways, including violence, harassment and discrimination.
Transsexual: Refers to a person who experiences a mismatch of the biological sex they were born
as and the biological sex they identify as. A transsexual sometimes undergoes medical treatment to
change his/her physical sex to match his/her sex identity through hormone treatments and/or
surgically. [Transexuals are included in the umbrella term transgender, but not all transgendered
people are transsexual. See also gender, sex, transgender.]
Transvestite/Cross Dresser: Individuals who regularly or occasionally wear the clothing socially
assigned to a gender not their own, but are usually comfortable with their anatomy and do not wish
to change it (i.e. they are not transsexuals). Cross-dresser is the preferred term for men who enjoy
or prefer women’s clothing and social roles. Contrary to popular belief, the overwhelming majority
of male cross-dressers identify as straight and often are married. Very few women call themselves
cross dressers.
Triangle: A symbol of remembrance. Gay men in the Nazi concentration camps were forced to
wear the pink triangle as a designation of being homosexual. Women who did not conform to
social roles, often believed to be lesbians, had to wear the black triangle. The triangles are worn
today as symbols of freedom, reminding us to never forget. """

In [40]:
liberal_terms=[]
for x in liberal_string.split('\n'):
    if len(x.split(":"))>1:
        liberal_terms.append(x.split(":")[0])

In [200]:
url='http://bellebrita.com/2014/08/feminism-101-learning-lingo/'
r=requests.get(url)
soup=BeautifulSoup.BeautifulSoup(r.text)
for x in soup.findAll('p')[12:-44]:
    x=x.text
    x=x.split(':')[0]
    liberal_terms.append(x)

In [227]:
df=pd.DataFrame()

In [ ]:
df.to_json

In [215]:
created_list=[liberal_terms,environmental,gaming,drugs,marxism,anarchy,fitness,sports,theological,technical_terms,\
    conservative_terms,controversial_terms,weeaboo]

In [228]:
for x,y in zip(created_list,['liberal_terms','environmental','gaming','drugs','marxism','anarchy','fitness','sports',\
            'theological','technical_terms','conservative_terms','controversial_terms','weeaboo']):
    json_name='/Users/thomas/GA-DSI/projects/projects-capstone/Work 2, Personalities/Key Terms Lists/'+y+'.json'
    df=pd.DataFrame(x,columns=['Terms'])
    json_file=df.to_json()
    with open(json_name, 'w') as fp:
        json.dump(json_file, fp)